# Chapter 2. Bayes's Theorem
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap02.html)

$P(A|B) = \frac{P(A)P(B|A)}{P(B)}$

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.